In [ ]:
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, make_scorer
from sklearn.pipeline import Pipeline 
from yellowbrick.model_selection import ValidationCurve, LearningCurve
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import numpy as np 
import xgboost as xgb
from matplotlib.offsetbox import AnchoredText
import heapq
import math 
import statistics 
from nasspython.nass_api import nass_data
from NdviApi import NDVI
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

In [ ]:
plt.style.use('ggplot')

In [ ]:
""" 
Helpful Links:
https://geo.fas.usda.gov/GADAS/index.html#
https://glam1.gsfc.nasa.gov/
https://www.mdpi.com/2072-4292/13/21/4227
https://glam1.gsfc.nasa.gov/api/doc/db/versions#default-db
"""

In [ ]:

# Generic function to initialize dataframes from USDA Quickstats data
def initialize_df(states, week, start_yr, end_yr, freq, suffix, var):
    loop = True 
    while loop:
        try:
            temp = pd.DataFrame(nass_data("B3744D45-DFBE-3B88-AFB1-25CBC8E64550", agg_level_desc="STATE", freq_desc=freq, short_desc=var)['data'])
            national_data = pd.DataFrame(nass_data("B3744D45-DFBE-3B88-AFB1-25CBC8E64550", agg_level_desc="NATIONAL", freq_desc=freq, short_desc=var)['data'])
            loop = False
        except ValueError as e:
            continue

    temp = pd.concat([temp, national_data], axis=0)
    temp = temp[temp["state_name"].isin(states)]
    if (freq == "WEEKLY"):
        temp = temp[temp["reference_period_desc"].eq(f"WEEK #{week}")]
    elif (freq == "ANNUAL"):
        temp = temp[temp["reference_period_desc"].eq("YEAR")]
    temp = temp[temp["year"].ge(start_yr) & temp["year"].le(end_yr)]
    temp = temp.pivot(index='year', columns='state_name', values="Value")
    temp = temp.add_suffix(suffix)
    temp = temp.replace(",", "", regex=True)
    temp = temp.fillna(0).astype(float)
    return temp 

In [ ]:

# NDVI data via https://glam1.gsfc.nasa.gov/
def get_ndvi_data(states, start_yr, end_yr, ids, mask, threshold, index, forecast=False):
    acc_ndvi_df = pd.DataFrame(index=index)
    for i in range(len(ids)):
        acc_ndvi = list()
        df = NDVI().get_data(
                            version='v15', 
                            sat='MOD',
                            mask=mask, 
                            shape='ADM',
                            start_yr=start_yr,
                            end_yr=end_yr,
                            start_month=1,
                            num_months=12,
                            ids=ids[i],
                            ts_type='cumulative',
                            mcv=0.0
                            )
        count=0
        df.reset_index(inplace=True)
        for j in range(df.shape[0]):
            if not forecast:
                if (int(df.loc[j, "ORDINAL DATE"][-3:]) == 329):
                        acc_ndvi.append(count)
                        count=0
                elif (df.loc[j, "SAMPLE VALUE"] >= threshold): 
                        count+=df.loc[j, "SAMPLE VALUE"] 
            else: 
                if df.loc[j, "SAMPLE VALUE"] >= threshold:
                    count+=df.loc[j, "SAMPLE VALUE"] 
        
        if forecast:
            acc_ndvi.append(count)
            
        acc_ndvi_df[states[i] + " NDVI"] = acc_ndvi  
                
    return acc_ndvi_df

In [ ]:

# Drought index data from https://droughtmonitor.unl.edu/About.aspx
def get_drought_data(states, ids, start_yr, end_yr, index):
    drought_data = pd.DataFrame(index=index)
    for i in range (len(ids)):
        if len(str(ids[i])) == 1:
            df = pd.read_json(f"https://usdmdataservices.unl.edu/api/StateStatistics/GetDroughtSeverityStatisticsByArea?aoi=0{ids[i]}&startdate=1/1/{start_yr}&enddate=1/1/{end_yr+1}&statisticsType=1")
        else:
            df = pd.read_json(f"https://usdmdataservices.unl.edu/api/StateStatistics/GetDroughtSeverityStatisticsByArea?aoi={ids[i]}&startdate=1/1/{start_yr}&enddate=1/1/{end_yr+1}&statisticsType=1")
        df = df.loc[:, ["ValidStart", "None", "D0", "D1", "D2", "D3", "D4"]]
        df["ValidStart"] = pd.to_datetime(df["ValidStart"]).dt.to_period('Y')
        df = df.iloc[::-1].set_index("ValidStart", drop=True)
        df = df.replace(",", "", regex=True)
        df = df.fillna(0).astype(float)
        df = df.groupby(lambda x: x.year)[["None", "D0", "D1", "D2", "D3", "D4"]].mean()
        df.index.name = None
        df = df.add_prefix(f"{states[i]} ")
        drought_data = pd.concat([drought_data, df], axis=1)
    return drought_data

In [ ]:
"""
Function to initialize data

Pass crop, week to take data from, and array of 1s and 0s indicating which additional variables to include.

Available crops are "CORN", "WHEAT, WINTER" and "SOYBEAN"

Below are the available variables, in order:
1. PCT Excellent at current week
2. PCT Good at current week
3. PCT Change in Excellent (pct @ week - pct @ week - 10)
4. PCT Change in Good (pct @ week - pct @ week - 10)
5. Acres planted/year
6. Acres harvested at current week
7. PCT Mature at current week
8. Drought data
9. PCT Fair at current week
10. PCT Poor at current week
11. PCT Very Poor at current week


Sample input below
"""


def get_data(crop, week, start_yr=2000, end_yr=2023, var_arr=[], forecast=False):
    
    if (crop == "CORN"):
        mask = "USDA-NASS-CDL_2018-2023_corn-50pp"
        states = ["US TOTAL", "WISCONSIN", "SOUTH DAKOTA", "OHIO", "NEBRASKA", "MISSOURI", "MINNESOTA", "KANSAS", "IOWA", "INDIANA", "ILLINOIS"]
        ndvi_ids = [27258, 26264, 26237, 26258, 26228, 26253, 26251, 26226, 26246, 26245, 26244]
        drought_ids = [55, 46, 39, 31, 29, 27, 20, 19, 18, 17]
        threshold = 0.58
    elif (crop == "SOYBEANS"):
        mask = "USDA-NASS-CDL_2018-2023_soybean-50pp"
        states = ["US TOTAL", "SOUTH DAKOTA", "OHIO", "NORTH DAKOTA", "NEBRASKA", "MISSOURI", "MINNESOTA", "IOWA", "INDIANA", "ILLINOIS", "ARKANSAS"]
        ndvi_ids = [27258, 26237, 26258, 26236, 26228, 26253, 26251, 26246, 26245, 26244, 26240]
        drought_ids = [46, 39, 38, 31, 29, 27, 19, 18, 17, 5]
        threshold = 0.58
    elif (crop == "WHEAT, WINTER"):
        mask = "USDA-NASS-CDL_2018-2023_winterwheat-50pp"
        states = ["US TOTAL", "WASHINGTON", "SOUTH DAKOTA", "OKLAHOMA", "NEBRASKA", "MONTANA", "MISSOURI", "KANSAS", "ILLINOIS", "IDAHO", "COLORADO"]
        ndvi_ids = [27258, 26234, 26237, 26230, 26228, 26227, 26253, 26226, 26244, 26225, 26224]
        drought_ids = [53, 46, 40, 31, 30, 29, 20, 17, 16, 8]
        threshold = 0.34
    
    idx = list(range(start_yr, end_yr + 1))
    """
    Get crop mask from: https://glam1.gsfc.nasa.gov/api/doc/db/versions#default-db
    """
    df = get_ndvi_data(states, start_yr, end_yr, ndvi_ids, mask=mask, threshold=threshold, index=idx, forecast=forecast)
    df.index.name = 'year'

    if not forecast:
        if (crop == "CORN"):
            yields = initialize_df(states, None, start_yr, end_yr, "ANNUAL", "", "CORN, GRAIN - YIELD, MEASURED IN BU / ACRE")
        else:
            yields = initialize_df(states, None, start_yr, end_yr, "ANNUAL", "", f"{crop} - YIELD, MEASURED IN BU / ACRE")

    overlay = pd.DataFrame()
    
    additional_vars = {
                    0: f"{crop} - CONDITION, MEASURED IN PCT EXCELLENT",
                    1: f"{crop} - CONDITION, MEASURED IN PCT GOOD",
                    2: f"{crop} - CONDITION, MEASURED IN PCT EXCELLENT",
                    3: f"{crop} - CONDITION, MEASURED IN PCT GOOD",
                    4: "PCT CHNGE EXCELLENT",
                    5: "PCT CHNGE GOOD",
                    6: f"{crop} - ACRES PLANTED",
                    7: f"{crop} - PROGRESS, MEASURED IN PCT HARVESTED",
                    8: f"{crop} - PROGRESS, MEASURED IN PCT EMERGED",

                    # Drought data not working atm
                    9: "Drought data" ,
                    10: f"{crop} - CONDITION, MEASURED IN PCT FAIR",
                    11: f"{crop} - CONDITION, MEASURED IN PCT POOR",
                    12: f"{crop} - CONDITION, MEASURED IN PCT VERY POOR",
    }

    for i in range(11):
        if var_arr[i] == 1:
            if i == 2:
                excellent_prev = initialize_df(states, week - 5, start_yr, end_yr, "WEEKLY", " EXCLNT CHNGE", additional_vars[i])
                excellent_curr = initialize_df(states, week, start_yr, end_yr, "WEEKLY", " EXCLNT CHNGE", additional_vars[i])
                pct_chnge_excellent = excellent_curr - excellent_prev
                overlay = pd.concat([overlay, pct_chnge_excellent], axis=1)
            elif i == 3:
                good_prev = initialize_df(states, week - 5, start_yr, end_yr, "WEEKLY", " GD CHNGE", additional_vars[i])
                good_curr = initialize_df(states, week, start_yr, end_yr, "WEEKLY", " GD CHNGE", additional_vars[i])
                pct_chnge_good = good_curr - good_prev
                overlay = pd.concat([overlay, pct_chnge_good], axis=1)
            elif i == 4:
                planted = initialize_df(states, None, start_yr, end_yr, "ANNUAL", " ACRES PLANTED", additional_vars[6])
                overlay = pd.concat([overlay, planted], axis=1)
            elif i == 5:
                if (crop == "CORN"):
                    harvested = initialize_df(states, week, start_yr, end_yr, "WEEKLY", " ACRES HARVESTED", "CORN, GRAIN - PROGRESS, MEASURED IN PCT HARVESTED")
                else:
                    harvested = initialize_df(states, week, start_yr, end_yr, "WEEKLY", " ACRES HARVESTED", additional_vars[7])
                overlay = pd.concat([overlay, harvested], axis=1)
            elif i == 6:
                if crop == "CORN":
                    var = f"{crop} - PROGRESS, MEASURED IN PCT SILKING"
                    maturity = initialize_df(states, week, start_yr, end_yr, "WEEKLY", " PROGRESS", var)
                else:
                    maturity = initialize_df(states, week, start_yr, end_yr, "WEEKLY", " PROGRESS", additional_vars[8])
                overlay = pd.concat([overlay, maturity], axis=1)
            elif i == 7:
                drought_index = get_drought_data(states, drought_ids, start_yr, end_yr, idx)
                overlay = pd.concat([overlay, drought_index], axis=1)
            elif i == 8:
                fair = initialize_df(states, week, start_yr, end_yr, "WEEKLY", " PCT FAIR", additional_vars[10])
                overlay = pd.concat([overlay, fair], axis=1)
            elif i == 9:
                poor = initialize_df(states, week, start_yr, end_yr, "WEEKLY", " PCT POOR", additional_vars[11])
                overlay = pd.concat([overlay, poor], axis=1)
            elif i == 10:
                very_poor = initialize_df(states, week, start_yr, end_yr, "WEEKLY", " PCT VERY POOR", additional_vars[12])
                overlay = pd.concat([overlay, very_poor], axis=1)
            elif i == 0:
                temp = initialize_df(states, week, start_yr, end_yr, "WEEKLY", " PCT EXCELLENT", additional_vars[i])
                overlay = pd.concat([overlay, temp], axis=1)   
            elif i == 1:
                temp = initialize_df(states, week, start_yr, end_yr, "WEEKLY", " PCT GOOD", additional_vars[i])
                overlay = pd.concat([overlay, temp], axis=1)
                
    x = pd.concat([df, overlay], axis=1)
    x.fillna(0, inplace=True)
    x.head()
    if not forecast:
        return yields, x
    else:
        return x


In [ ]:
def pairplots():
    global x 
    global y 
    for state in y.columns:
        state_data = x.filter(regex=state)
        temp = pd.DataFrame(y[state], columns=[state])
        temp.reset_index(drop=True, inplace=True)
        g = sns.pairplot(state_data.assign(yields=temp[state]), y_vars=['yields'][:], x_vars=state_data.columns)
        sns.despine();

In [ ]:
# Optional filtering based on Spearman correlation

def spearman_correlations(filter=True):
    global x 
    global y 
    fig, axs = plt.subplots(5, 2, figsize=(18, 18))

    for i, ax in enumerate(fig.axes):
        state = y.columns[i-1]
        corr = x.filter(regex=state).apply(lambda x: spearmanr(x, y[state])[0])
        if filter:
            temp = abs(corr)
            x.drop(columns=temp.nsmallest(math.floor(len(corr.index) / 2)).index, inplace=True)
        ax.barh(y=corr.index.str.lstrip(state), width=corr.sort_values(), color='lightsteelblue')
        ax.set_title(state);
    
    plt.subplots_adjust(hspace=0.5, wspace=0.4)
    plt.show()

In [ ]:
# Optional filtering based on mutual information
def mutual_information(filter=True):
    fig, axs = plt.subplots(5, 2, figsize=(18, 18))

    for i, ax in enumerate(fig.axes):
        state = y.columns[i-1]
        state_data = x.filter(regex=state)
        mi = mutual_info_regression(X=state_data,
                                    y=y[state],
                                    random_state=42
                                    )
        mutual_info = pd.DataFrame(mi, index=state_data.columns, columns=[state])
        if filter:
            x.drop(columns=mutual_info[state].nsmallest(math.floor(len(mutual_info.index) / 2)).index, inplace=True)
        ax.barh(state_data.columns, mutual_info[state])
        ax.set_title(state)
    plt.tight_layout()


In [ ]:

# Returns feature coefficients of regression model
def feature_importance(model, df, ax, num_features, plot=False, print_results=False):
        importance = model.coef_
        keys = list(df.keys())
        abs_weights = {}
        weights = {}
        for i,v in enumerate(importance):
            if print_results:
                print("Feature: %s, Score: %.5f" % (keys[i],v))
            abs_weights[keys[i]] = abs(v)
            weights[keys[i]] = v
        if plot:
            ax.bar([x for x in range(len(importance))], height=importance, color='b')
            tickvals = range(0, len(importance))
            cols = df.columns
            ax.set_xticks(ticks=tickvals, labels=cols, rotation=45, fontsize='xx-small', fontstretch='extra-condensed')
        
        largest_features = heapq.nlargest(num_features, abs_weights, key=abs_weights.get)

        avg_abs_weight = sum([abs_weights.get(key) for key in largest_features]) / len(largest_features)
        return largest_features, weights, avg_abs_weight

In [ ]:

# Lasso and xgb finetuning

def get_best_model(X, y, kwargs=None, lasso=True, print_results=False, scoring=None, cv=5):
    xgb_params = {
    "learning_rate": np.arange(0.01, 0.2, 0.01),
    "min_child_weight": np.arange(1, 5, 1),
    "n_estimators": range(800, 1200),
    "max_depth": range(1, 5),
    "colsample_bytree": np.arange(0.1, 1, 0.1),
    "subsample": np.arange(0.1, 1, 0.1)
    }

    lasso_params = {
        "alpha": np.arange(0.05, 2, 0.05)
    }

    if not lasso:
        optimized_model = RandomizedSearchCV(param_distributions=xgb_params, estimator=xgb.XGBRegressor(), scoring=scoring, verbose=1, random_state=42, cv=2)
        optimized_model.fit(X, y)
        if print_results:
            print("Best Parameters:", optimized_model.best_params_)
    else: 
        optimized_model = RandomizedSearchCV(param_distributions=lasso_params, estimator=Lasso(random_state=42), scoring=scoring, verbose=1, random_state=42, cv=cv)
        optimized_model.fit(X, y)
        if print_results:
            print("Best Parameters:", optimized_model.best_params_)
    return optimized_model

In [ ]:
# Fitting models, analyzing and plotting data

def reg_plot(x, y, title, ax=None, forecast=False, x_forecast=None, model_df=None):

    if forecast:
        x_forecast = x_forecast.filter(axis=1, items=x.columns)
        for i in x.columns:
            if i not in x_forecast.columns:
                x.drop(columns=[i], inplace=True)
    
    models = {
    'Linear Regression': LinearRegression(),
    'Lasso' : Lasso(alpha = 0.45, random_state=42),
    'Ridge' : Ridge(random_state=42),
    'Decision Tree': DecisionTreeRegressor(max_depth=3, random_state=42),
    'KNN': KNeighborsRegressor(),
    'XGB': xgb.XGBRegressor(n_estimators=1000,
                            max_depth=3,
                            gamma = 5,
                            subsample = 0.5,
                            random_state=42),
    'XGBRF': xgb.XGBRFRegressor(n_estimators=1000, 
                                subsample = 0.5,
                                random_state=42),
    'RF Regressor': RandomForestRegressor(
                                        random_state=42)
    }
    
    r2_scores = {}
    maes = {}
    print(title)
    for i in range(25):
        X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state = i, shuffle=True)
        for name, md in models.items():
            md.fit(X_train,y_train)
            y_pred = md.predict(X_test)
            print(f"{name}: mae : {mean_absolute_error(y_test, y_pred)} score : {r2_score(y_test, y_pred)}")
            if i == 0:
                maes[md] = mean_absolute_error(y_test, y_pred)
                r2_scores[md] = r2_score(y_test, y_pred)
            else:
                maes[md] = maes.get(md) + mean_absolute_error(y_test, y_pred)
                r2_scores[md] = r2_scores.get(md) + r2_score(y_test, y_pred)
            model_df.loc[title, name] = r2_score(y_test, y_pred)
    print("-------------")

    for md in r2_scores.keys():
        r2_scores[md] = r2_scores.get(md) / 25
        maes[md] = maes.get(md) / 25
    model = max(r2_scores, key=lambda x:r2_scores[x])

    model.fit(X_train, y_train)
    y_train_pred = pd.DataFrame(model.predict(X_train), index=X_train.index)
    y_test_pred = pd.DataFrame(model.predict(X_test), index=X_test.index)

    if forecast:
        y_forecast = model.predict(x_forecast)
        ax.scatter(24, y_forecast, color='purple')

    combinedX = pd.concat([X_train, X_test], axis=0).sort_index()
    combinedY = pd.concat([y_train, y_test], axis=0).sort_index()
    combinedY_pred = pd.concat([y_train_pred, y_test_pred], axis=0).sort_index()
    ax.plot(combinedX.index, combinedY, label='Actual Yield')
    ax.plot(combinedX.index, combinedY_pred, label='Predicted Yield')
    ax.scatter(X_test.index, y_test_pred, label='Out of Sample Predictions', color='black')
    ax.legend(loc='lower right', fontsize='small')
    
    ax.set_title(title)

    s = r2_scores[model]
    r = maes[model]
    best_model = [i for i in models if models[i] == model]
    at = AnchoredText(
        f"OOS AVG R2: {s:.2f}\nOOS AVG MAE {r:.2f}\nModel: " + best_model[0],
        prop=dict(size="medium"),
        frameon=True,
        loc="upper left",
    )
    at.patch.set_boxstyle("square, pad=0.0")
    ax.add_artist(at)
    if forecast:
        return ax, s, y_forecast, model_df
    else:
        return ax, s, model_df

In [ ]:
def rmse(y_true, pred):
    return np.sqrt(mean_squared_error(y_true=y_true, y_pred=pred))

In [ ]:

def cross_val_curves(x, y, ax):
    X_train, _, y_train, _ = train_test_split(x,y, test_size = 0.1, random_state = 42, shuffle=True)

    rmse_score = make_scorer(rmse)
    cv_rmse = {}
    alphas = list(np.arange(0.1, 3.0, 0.1))
    for alpha in alphas:
        pipe = Pipeline([('scaler', MinMaxScaler()),
                        ('lasso', Lasso(alpha=alpha))])
        cv_rmse[alpha] = cross_val_score(pipe,
                                        X=X_train,
                                        y=y_train,
                                        scoring=rmse_score,
                                        cv=5)


    cv_rmse = pd.DataFrame.from_dict(cv_rmse, orient='index')
    best_alpha, best_rmse = cv_rmse.mean(1).idxmin(), cv_rmse.mean(1).min()
    cv_rmse = cv_rmse.stack().reset_index()
    cv_rmse.columns=['alpha', 'iter', 'RMSE']
    ax = sns.lineplot(x='alpha', y='RMSE', data=cv_rmse, ax=ax)
    ax.set_title(f'Cross-Validation Results Lasso | Best alpha: {best_alpha} | Best RMSE: {best_rmse:.2f}');

In [ ]:
def cv_curves():
    fig, axs = plt.subplots(5, 2, figsize=(18, 18))
    for i, ax in enumerate(fig.axes):
        state = y.columns[i]
        cross_val_curves(x.filter(regex=state), y[state], ax=ax)
    plt.subplots_adjust(hspace=0.5)

In [ ]:
fig, axs = plt.subplots(6, 2, figsize=(20, 20))
fig.delaxes(axs[5, 1])

r2_by_state = {}
cumulative_r2 = {}
y, x = get_data("WHEAT, WINTER", 23, 2000, 2023, [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])
model_df = pd.DataFrame(index=y.columns, columns=["Linear Regression", "Lasso", "Ridge", "Decision Tree", "KNN", "XGB", "XGBRF", "RF Regressor"])
preds = pd.DataFrame(index=["WHEAT, WINTER"], columns = y.columns)
mutual_information(filter=True)
for i, ax in enumerate(fig.axes):
        col_name = y.columns[i]
        input = x.filter(regex=col_name)
        x_forecast = get_data("WHEAT, WINTER", 23, 2024, 2024, [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1], forecast=True)
        _, r2_by_state[col_name], y_forecast, model_df = reg_plot(input, y[col_name], col_name, ax, forecast=True, x_forecast=x_forecast.filter(regex=col_name), model_df=model_df)
        preds.loc["WHEAT, WINTER", col_name] = y_forecast
plt.subplots_adjust(hspace=0.7)
plt.show()

preds.to_csv("winter wheat 2024.csv")
model_df.to_csv("winter wheat models r2.csv")




In [ ]:
fig, axs = plt.subplots(6, 2, figsize=(18, 18))
fig.delaxes(axs[5, 1])

r2_by_state = {}
cumulative_r2 = {}



y, x = get_data("CORN", 30, 2000, 2023, [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])
model_df = pd.DataFrame(index=y.columns, columns=["Linear Regression", "Lasso", "Ridge", "Decision Tree", "KNN", "XGB", "XGBRF", "RF Regressor"])
preds = pd.DataFrame(index=["CORN"], columns=y.columns)
mutual_information(filter=True)
for i, ax in enumerate(fig.axes):
        if i <= 9:
                col_name = y.columns[i]
                input = x.filter(regex=col_name)
                x_forecast = get_data("CORN", 30, 2024, 2024, [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1], forecast=True)
                _, r2_by_state[col_name], y_forecast, model_df= reg_plot(input, y[col_name], col_name, ax, forecast=True, x_forecast=x_forecast.filter(regex=col_name), model_df=model_df)
                preds.loc["CORN", col_name] = y_forecast
plt.subplots_adjust(hspace=0.7)
plt.show()

preds.to_csv("corn 2024.csv")
model_df.to_csv("corn models r2.csv")


In [ ]:
fig, axs = plt.subplots(6, 2, figsize=(18, 18))
fig.delaxes(axs[5, 1])

r2_by_state = {}
cumulative_r2 = {}
y, x = get_data("SOYBEANS", 30, 2000, 2023, [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])
model_df = pd.DataFrame(index=y.columns, columns=["Linear Regression", "Lasso", "Ridge", "Decision Tree", "KNN", "XGB", "XGBRF", "RF Regressor"])
preds = pd.DataFrame(index=["SOYBEANS"], columns=y.columns)
mutual_information(filter=True)
for i, ax in enumerate(fig.axes):
        if i <= 9:
                col_name = y.columns[i]
                input = x.filter(regex=col_name)
                x_forecast = get_data("SOYBEANS", 30, 2024, 2024, [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1], forecast=True)
                _, r2_by_state[col_name], y_forecast, model_df = reg_plot(input, y[col_name], col_name, ax, forecast=True, x_forecast=x_forecast.filter(regex=col_name), model_df=model_df)
                preds.loc["SOYBEANS", col_name] = y_forecast
plt.subplots_adjust(hspace=0.7)
plt.show()

preds.to_csv("soybeans 2024.csv")
model_df.to_csv("soybean models r2.csv")